In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import pydot
keras.utils.vis_utils.pydot = pydot
from keras import layers

In [2]:
device_name = tf.test.gpu_device_name()
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [7]:
# Read data and preprocess
df = pd.read_csv("data/rankings_100.csv")

df_labels = df.pop("score")
inputs = {}

for name, column in df.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(df[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)
preprocessed_inputs = [all_numeric_inputs]

for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  lookup = layers.StringLookup(vocabulary=np.unique(df[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

df_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

df_features_dict = {name: np.array(value) 
                         for name, value in df.items()}

features_dict = {name:values[:1] for name, values in df_features_dict.items()}
df_preprocessing(features_dict)


In [8]:
def mythic_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam())
  return model

mythic_model = mythic_model(df_preprocessing, inputs)


,id,dungeon_name,dungeon_num_bosses,mythic_level,time_remaining_ms,weekly_modifiers,region,roster
0,7620658,The Underrot,4,26,276062,"['Fortified', 'Afflicted', 'Raging']",Europe,"[{'role': 'dps', 'class': 'Priest-Shadow', 'ra..."
1,7313051,The Underrot,4,26,218493,"['Fortified', 'Afflicted', 'Raging']",United States & Oceania,"[{'role': 'dps', 'class': 'Priest-Shadow', 'ra..."
2,7238696,The Underrot,4,26,202799,"['Fortified', 'Afflicted', 'Raging']",Europe,"[{'role': 'tank', 'class': 'Paladin-Protection..."
3,7215746,The Underrot,4,26,195531,"['Fortified', 'Afflicted', 'Raging']",Europe,"[{'role': 'tank', 'class': 'Paladin-Protection..."
4,7778214,The Underrot,4,26,180974,"['Fortified', 'Afflicted', 'Raging']",Europe,"[{'role': 'dps', 'class': 'Warrior-Fury', 'rac..."


In [ ]:
mythic_model.fit(x=df_features_dict, y=df_labels, epochs=10)